# Homework 03. Jeehee Kim (Student ID 20205098)

In [1]:
# Read a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

# Make the fuction for transfer dictionary format to fasta format
def w_fasta(fa_filename, g_dict):
    fa_F = open(fa_filename, 'w')
    for g_hd,g_sq in g_dict.items():
        fa_F.write(">"+g_hd+"\n"+g_sq+"\n")        
    fa_F.close()
    
###########################################################################   
## 1. Read GCF_009858895.2 genomic info.

GCF_f = '../data/GCF_009858895.2_ASM985889v3_genomic.fna'

Ref_seq = read_fasta(GCF_f)



In [2]:
## 2. Make N-protein cDNA sequence.
### N-protein index in GCF_009858895.2
f_ref_idx = '../data/GCF_009858895.2_ASM985889v3_genomic.gff'

def read_gff(tmp_filename, gene_name):
    f = open(tmp_filename, 'r')
    for line in f.readlines():
        if 'gene' in line and "gene={}".format(gene_name) in line:
            return int(line.split()[3])-1, int(line.split()[4])-1
        

print(read_gff(f_ref_idx, 'N'))
### Get N-protein cDNA in fasta form

Idx_N = read_gff(f_ref_idx, 'N')
print(' ')

for seq in Ref_seq.values():
    ref_N_seq = seq[Idx_N[0] : Idx_N[1]+1]
ref_N_cDNA ={'N_protein': ref_N_seq}
#print(ref_N_cDNA)
print('N protein: ', ref_N_cDNA['N_protein'][:30])

w_fasta("ref_N_cDNA.fa", ref_N_cDNA)

(28273, 29532)
 
N protein:  ATGTCTGATAATGGACCCCAAAATCAGCGA


In [3]:
## 3. Translate it to protein sequence.  
#transeq --sequence ../02_PCR/ref_N_cDNA.fa --outseq ref_N_protein.fa

ref_N_protein = []

ref_P = read_fasta('ref_N_protein.fa')
for H_P,S_P in ref_P.items():
    ref_N_protein.append(S_P)

#print(ref_N_protein[0])

###########################################################################
## Q1. Compare it to GCF_009858895.2_ASM985889v3_protein.fna. Are they same?
Sample_protein = '../data/GCF_009858895.2_ASM985889v3_protein.faa'
sampleP_file = read_fasta(Sample_protein)
#print(sampleP_file)

Sample_N_protein = []
for h_P, s_P in sampleP_file.items():
    if 'nucleocapsid phosphoprotein' in h_P:
        #print(s_P)
        Sample_N_protein.append(s_P)
        

if ref_N_protein[0][:-1] == Sample_N_protein[0]:     # [:-1] ; for stop codon
    print('They are same!')
else:
    print('No!!!!!')

They are same!


In [4]:
## 4. Based on this info, extract N proteins cDNA/proteins from other SARS-CoV-2 genomes (either 02_03 or 03_27).
## Compare reference N protein cDNA with SARS-CoV-genome using by blastn
#makeblastdb -dbtype nucl -in ../data/2019nCoV_genomes.2020_03_27.fa -out 2019nCoV_genomes.2020_03_27.blastnb.nbl
#blastn -db 2019nCoV_genomes.2020_03_27.blastnb.nbl -query ref_N_cDNA.fa -outfmt 7 -out cDNA_compare.nblast.nbl

### Extract N protein CDS form 2019nCoV_genomes
g200327_f = '../data/2019nCoV_genomes.2020_03_27.fa'
rd_g200327_dict = read_fasta(g200327_f)

def Extr_P_seq(nblasted_file,query,seq_dict):
    protein_idx = dict()
    f = open(nblasted_file, 'r')
    for line in f.readlines():
        if line.startswith(query):
            s_h = str(line.split()[1])
            idx = (int(line.split()[8])-1), (int(line.split()[9])-1)            
            protein_idx[s_h]=seq_dict[s_h][idx[0]:idx[1]+1] 
    return protein_idx
   
    
k = './cDNA_compare.nblast.nbl'
s_list = '../data/2019nCoV_genomes.2020_03_27.fa'

S = read_fasta(s_list)
s_dict = dict()
for H0327, S0327 in S.items():
    s_dict.setdefault(H0327[:8],S0327)

Nseq_dict = Extr_P_seq( k , 'N_protein' , s_dict)
#print(s_dict)
#print(Nseq_dict)


t = w_fasta('./200327_N_seq.fa', Nseq_dict)

## Translate it to protein sequence.
#transeq --sequence 200327_N_seq.fa --outseq 200327_N_AA.fa
## Compare ref_N_protein amino acid with SARS-CoV-AA using by blastp
#makeblastdb -dbtype prot -in 200327_N_AA.fa -out 200327_N_protein.pblast
#blastp -db 200327_N_protein.pblast -query ref_N_protein.fa -outfmt 7 -out compare_200327SARS_CoV_2.blastp.tbl

def read_blast(blasted_file,query):
    f = open(blasted_file,'r')
    for line in f.readlines():
        if line.startswith(query):
            if float(line.split()[2]) == 100.000 :
                s_name = str(line.split()[1])
                print("Perpectly matched!", s_name)
            else:
                s_name = str(line.split()[1])
                Pcnt = str(line.split()[2]) 
                mismatch = str(line.split()[4])
                print(s_name, Pcnt, "% matched.",mismatch,"mismatches")

read_blast("./compare_200327SARS_CoV_2.blastp.tbl", "N_protein_1")
print(' ')

## Similarity checking
def blast_similarity(blasted_file,query):
    f = open(blasted_file,'r')
    percentage_list = []
    for line in f.readlines():
        if line.startswith(query):
            percentage_list.append(float(line.split()[2]))
    max_pcntg = max(percentage_list)
    min_pcntg = min(percentage_list)
    f.close()
    
    f = open(blasted_file,'r')
    max_pcntg_name = []
    min_pcntg_name = []
    for line in f.readlines():
        if line.startswith(query):
            if float(line.split()[2]) == max_pcntg:
                max_pcntg_name.append(line.split()[1])
            elif float(line.split()[2]) == min_pcntg:
                min_pcntg_name.append(line.split()[1])
    f.close()        
    print( "Most matched:",max_pcntg, max_pcntg_name )
    print( "Most different:",min_pcntg, min_pcntg_name )
    
# Q2. Which pairs are most similar ones? Which pairs are most different ones?
print("SARS-CoV-2 genomes compare with N protein's cDNA")
blast_similarity('cDNA_compare.nblast.nbl','N_protein')
print(' ')
print("SARS-CoV-2 genomes compare with N protein's amino acid seq")
blast_similarity('./compare_200327SARS_CoV_2.blastp.tbl','N_protein_1')

Perpectly matched! MT246454_1
Perpectly matched! MT123291_1
Perpectly matched! MT123293_1
Perpectly matched! MT246449_1
Perpectly matched! MT246450_1
Perpectly matched! MT246452_1
Perpectly matched! MT246453_1
Perpectly matched! MT246455_1
Perpectly matched! MT246457_1
Perpectly matched! MT246458_1
Perpectly matched! MT246459_1
Perpectly matched! MT246460_1
Perpectly matched! MT246461_1
Perpectly matched! MT246462_1
Perpectly matched! MT246464_1
Perpectly matched! MT246466_1
Perpectly matched! MT246467_1
Perpectly matched! MT246468_1
Perpectly matched! MT246469_1
Perpectly matched! MT246471_1
Perpectly matched! MT246472_1
Perpectly matched! MT246473_1
Perpectly matched! MT246474_1
Perpectly matched! MT246475_1
Perpectly matched! MT246476_1
Perpectly matched! MT246477_1
Perpectly matched! MT246478_1
Perpectly matched! MT246479_1
Perpectly matched! MT246480_1
Perpectly matched! MT246481_1
Perpectly matched! MT246482_1
Perpectly matched! MT246484_1
Perpectly matched! MT246486_1
Perpectly 

In [5]:
# 5. Based on this info, extract N proteins cDNA/proteins from other SARS & MERS genomes.
#makeblastdb -dbtype nucl -in ../data/SARS_genomes.2020_02_03.fa -out SARS_genomes.2020_02_03.blastn.nbl
#blastn -db SARS_genomes.2020_02_03.blastn.nbl -query ref_N_cDNA.fa -outfmt 7 -out SARS_genome_compare.blastn.nbl

SARS_g = '../data/SARS_genomes.2020_02_03.fa'
#transeq --sequence ../data/SARS_genomes.2020_02_03.fa --outseq SARS_genome.2020_02_03_AA.fa -frame 6
#makeblastdb -dbtype prot -in ../data/SARS_genome.2020_02_03_AA.fa -out SARS_genome_200203.blastp
#blastp -db SARS_genome_200203.blastp -query ref_N_protein.fa -outfmt 7 -out compare_200203SARS.blastp.tbl

MERS_g = '../data/MERS_genomes.2020_02_03.fa'
#transeq --sequence ../data/MERS_genomes.2020_02_03.fa --outseq MERS_genome.2020_02_03_AA.fa -frame 6
#makeblastdb -dbtype prot -in MERS_genome.2020_02_03_AA.fa -out MERS_genome_200203.blastp
#blastp -db MERS_genome_200203.blastp -query ref_N_protein.fa -outfmt 7 -out compare_200203MERS.blastp.tbl


# Q3. Which protein is most similar to SARS-CoV N protein? Which protein is most different to MERS N protein?
print(' ')
print("SARS proteins compare with N protein's amino acid seq")
blast_similarity('compare_200203SARS.blastp.tbl','N_protein_1')

print(' ')
print("MERS proteins compare with N protein's amino acid seq")
blast_similarity('compare_200203MERS.blastp.tbl','N_protein_1')


 
SARS proteins compare with N protein's amino acid seq
Most matched: 90.544 ['KU973692.1_3', 'DQ898174.1_1', 'FJ882949.1_1', 'FJ882934.1_3', 'FJ882937.1_2', 'FJ882935.1_1', 'FJ882933.1_2', 'FJ882947.1_3', 'FJ882946.1_1', 'FJ882927.1_2', 'FJ882932.1_3', 'FJ882939.1_3', 'FJ882936.1_3', 'FJ882938.1_2', 'FJ882941.1_1', 'FJ882929.1_1', 'FJ882944.1_3', 'FJ882940.1_3', 'FJ882954.1_2', 'FJ882960.1_3', 'FJ882955.1_3', 'FJ882931.1_3', 'FJ882928.1_2', 'FJ882926.1_2', 'FJ882930.1_2', 'AY362698.1_1', 'AY394850.2_2']
Most different: 89.573 ['DQ084200.1_3', 'DQ022305.2_2']
 
MERS proteins compare with N protein's amino acid seq
Most matched: 50.142 ['KX574227.1_3']
Most different: 43.333 ['KX108941.1_2']
